<a href="https://colab.research.google.com/github/saishshinde15/Text_summarization/blob/main/Text_summrization_mywork1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
df=pd.read_csv('medquad.csv')

In [5]:
df

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
...,...,...,...,...
16407,What is (are) Diabetic Neuropathies: The Nerve...,Focal neuropathy appears suddenly and affects ...,NIDDK,Diabetic Neuropathies: The Nerve Damage of Dia...
16408,How to prevent Diabetic Neuropathies: The Nerv...,The best way to prevent neuropathy is to keep ...,NIDDK,Diabetic Neuropathies: The Nerve Damage of Dia...
16409,How to diagnose Diabetic Neuropathies: The Ner...,Doctors diagnose neuropathy on the basis of sy...,NIDDK,Diabetic Neuropathies: The Nerve Damage of Dia...
16410,What are the treatments for Diabetic Neuropath...,The first treatment step is to bring blood glu...,NIDDK,Diabetic Neuropathies: The Nerve Damage of Dia...


In [6]:
doc = df[df.focus_area == 'Glaucoma']['answer'].sample(random_state=42)

In [7]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)


In [8]:
print(wrap(doc.iloc[0]))

At first, open-angle glaucoma has no symptoms.  It causes no pain.
Vision seems normal.  Without treatment, people with glaucoma will
slowly lose their peripheral, or side vision.  They seem to be looking
through a tunnel.  Over time, straight-ahead vision may decrease until
no vision remains.


In [12]:
sents = nltk.sent_tokenize(doc.iloc[0])
for sent in sents:
  print(wrap(sent))


At first, open-angle glaucoma has no symptoms.
It causes no pain.
Vision seems normal.
Without treatment, people with glaucoma will slowly lose their
peripheral, or side vision.
They seem to be looking through a tunnel.
Over time, straight-ahead vision may decrease until no vision remains.


In [13]:
featurizer = TfidfVectorizer(
    stop_words=stopwords.words('english'),
    norm='l1',
)

In [14]:
X = featurizer.fit_transform(sents)

In [15]:
def get_sentence_score(tfidf_row):
  x = tfidf_row[tfidf_row != 0]
  return x.mean()

In [16]:
scores = np.zeros(len(sents))
for i in range(len(sents)):
  score = get_sentence_score(X[i,:])
  scores[i] = score

In [17]:
sort_idx = np.argsort(-scores)

In [18]:
print("Generated summary:")
for i in sort_idx[:5]:
  print(wrap("%.2f: %s" % (scores[i], sents[i]))) #top N sentences

Generated summary:
0.50: It causes no pain.
0.33: Vision seems normal.
0.33: They seem to be looking through a tunnel.
0.20: At first, open-angle glaucoma has no symptoms.
0.14: Over time, straight-ahead vision may decrease until no vision
remains.
